# Noisy Amplitude Estimation with low-depth algorithms

## Introduction 
This notebook deals with a low-depth algorithm to estimate the amplitude of a unitary operation $U$ on a noisy system following these articles:
* [[Tiron et al.]](https://arxiv.org/abs/2012.03348v1): Tudor Giurgica-Tiron, Iordanis Kerenidis, Farrokh Labib, Anupam Prakash, and William Zeng (2022), "Low depth algorithms for quantum amplitude estimation", _Quantum,_ Volume 6, page 745;  arXiv:2012.03348

* [[Tanaka et al.]](https://link.springer.com/article/10.1007/s11128-021-03215-9) Tomoki Tanaka, Yohichi Suzuki, Shumpei Uno, Rudy Raymond, Tamiya Onodera & Naoki Yamamoto (2021), "Amplitude estimation via maximum likelihood on noisy quantum computer", _Quantum Information Processing_, Volume 20, Issue 9, Article 293; arXiv:2006.16223

* [[Suzuki et al.]](http://arxiv.org/abs/1904.10246v2): 
Yohichi Suzuki, Shumpei Uno, Rudy Raymond, Tomoki Tanaka, Tamiya Onodera, and Naoki Yamamoto (2020): "Amplitude estimation without phase estimation", _Quantum Information Processing,_ Volume 19, Issue 2, Article 75; arXiv:1904.10246

* [[Brassard et al.]](https://arxiv.org/abs/quant-ph/0005055v1): Gilles Brassard, Peter Hoyer, Michele Mosca, and Alain Tapp (2002), "Quantum amplitude amplification and estimation", _Contemporary Mathematics_, Volume 305, pp. 53-74; arXiv:0005055v1

## Background

### 1. Amplitude Estimation

Given a unitary $U$ that acts on $(n+1)$ qubits such that $U:\ket{0}^n \ket{0} \mapsto \cos \theta\ \ket{\Psi_0}\ket{0} + \sin \theta \ket{\Psi_1} \ket{1}$ as an oracle with an unknown angle $\theta\in[0,\pi/2]$, the amplitude estimation problem aims to find an angle $\hat{\theta}$ such that $|\hat{\theta} - \theta| \leq \varepsilon$ for some precision parameter $\varepsilon$. Doing this classically, i.e. based on the probability $\sin^2(\theta)$,  would require calling the oracle $O(\frac{1}{\varepsilon^2})$ times. The quantum amplitude estimation algorithm uses the Quantum Fourier Transform as a sub-routine and estimates the amplitude using only $O(\frac{1}{\varepsilon})$ queries. However, obtaining this quadratic speedup requires a high circuit depth of $O(\frac{1}{\varepsilon})$ and in the presence of noise, the accuracy this circuit cannot be guaranteed.

On the other hand, by settling for a sub-quadratic speedup, it is possible to design quantum-classical hybrid algorithms that require only a constant depth on noisy systems. This class of algorithms use amplitude amplification followed by maximum likelihood estimation (MLE) to compute $\hat{\theta}$. This Q\# + Python notebook demonstrates one such low-depth algorithm following the work of [Tiron et al.](https://arxiv.org/abs/2012.03348v1), [Tanaka et al.](https://link.springer.com/article/10.1007/s11128-021-03215-9) and [Suzuki et al.](http://arxiv.org/abs/1904.10246v2). By adjusting the various parameters in this notebook it is possible to reproduce the Power Law based algorithm in the last reference as well as the Linear and Exponential schedule algorithms in the first one.

### 2. Maximum Likelihood Estimation

As the name suggests, Maximum Likelihood Estimation (MLE) aims to answer the question: *"Given a sequence of i.i.d samples from an unknown distribution parameterized by a variable, say $\alpha$, what value of $\alpha$ is the most likely to produce the observed samples?"*

Formally, for a random variable $X$, let $\text{Pr}(X, \alpha)$ be the density function parametrized by an unknown parameter $\alpha$. Now, given $n$ independent samples from this distribution such that $X_i = x_i$ for $i \in \{1,\dots,n\}$, the likelihood of $\alpha$ taking a value $\theta$ is given by the *likelihood function*:
$$ L(\theta|x) = \Pi_{i=1}^n \text{Pr} (X_i=x_i | \alpha = \theta). $$
For ease of notation, we will denote the likelihood function as $L(\theta)$ and the dependency on the samples $x_i$ is clear from context.
MLE suggests that the $\theta$ that maximizes the above equation is a good estimate for the ideal value $\theta^*$ actually taken by $\alpha$. To avoid working with small probabilities, it is common to instead maximize the *log-likelihood* function $\ell(\theta) = \log L(\theta)$. Then, a good estimate for alpha will be:
$$ \hat{\theta} = \arg \max_{\theta} L(\theta) = \arg \max_{\theta} \ell(\theta).$$


## Setup

First, we import the requisite packages, load the Q# environment, and the necessary Q# libraries needed to run the code.

In [ ]:
import numpy as np
from numpy import *
import math
import traceback
import sys

In [ ]:
import qsharp

In [ ]:
%%qsharp
open Microsoft.Quantum.Random;
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Measurement;
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Diagnostics;

## Amplitude Amplification: a refresher

For an unknown angle $\theta^* \in [0,\pi/2]$, define the unitary $U$ acting on $(n+1)$ qubits with the action
$$ U: \ket{0}^n \ket{0} \mapsto \ket{\Psi} := \sin \theta^* \ket{\Psi_1}\ket{1} + \cos \theta^* \ket{\Psi_0}\ket{0}$$
where $\ket{\Psi_1}$ labeled as a *good* state. 
In this sample, we set the default angle as $\theta^* = \pi / (2 \cdot \Phi) \approx 0.9708$ where $\Phi = (1 + \sqrt{5})/2$ is the Golden Ratio.

In [ ]:
%%qsharp
/// # Summary
/// Returns the value of the unknown angle.
/// Angle value should be in [0, PI()/2] 
function UnknownTHETA(): Double {
    return PI()/(1.0 + Sqrt(5.0));
}

📝 **Remark:** Modify the value of the angle in the previous cell to try the sample for different values of $\theta^*$.


The following operation describes one such unitary acting on $2$ qubits applying an arbitrary rotation for a fixed angle $\theta^*$. 

In [ ]:
%%qsharp
/// # Summary 
/// The U operation. 
/// Unitary takes |0>|0> -> cos theta* |0>|0> + sin theta* |1>|1>
/// This operation is accomplished by applying Ry(-2*theta)
///
/// # Inputs
/// ## qs
/// The 2 qubits register on which the oracle is applied.
operation ArbitRotation (qs : Qubit[]) : Unit is Adj + Ctl {
    
    Ry(2.0*UnknownTHETA(), qs[0]); // Takes |0>|0> -> cos theta* |0>|0> + sin theta* |1>|0>
    CNOT(qs[0], qs[1]); // Takes cos theta* |0>|0> + sin theta* |1>|0> -> cos theta* |0>|0> + sin theta* |1>|1>
}

📝 **Remark:** To run the sample for a different unitary operation, modify the above cell accordingly.

By repeatedly measuring the state $\ket{\Psi}$ and counting the fraction of $1$ outcomes would give an estimate for $\sin^2 \theta^*$. While this is a low-depth circuit, it does not provide any speedup over classical methods. 

However, using quantum amplitude amplification, it is possible to *amplify* the probability of seeing a $1$ outcome. Then, fewer queries can be used to obtain a good estimate for $\sin^2 \theta^*$ and, by extension, $\theta^*$. This is done by applying the operator 
$$ \mathbf{Q} = - U S_0 U^{\dagger} S_{\chi} $$
where $S_{\chi}$ marks the *good* states and $S_0$ marks the all-zeroes state. In other words, 
$ S_{\chi} \ket{\Psi_1}\ket{1} = -  \ket{\Psi_1}\ket{1}$ and other states remain unchanged while $S_{0} \ket{0}^{n+1} = - \ket{0}^{n+1}$ and other states remain unchanged. The following operations describe how to construct the amplification operator $\mathbf{Q}$ by first constructing the $S_0$ and $S_{\chi}$ operations.

In [ ]:
%%qsharp
/// #Summary 
/// The S_0 operation. Applies -1 to |0>|0> state and nothing to others. 
/// Can be easily extended to act on qubit registers of any length.
///
/// #Inputs
/// ## register
/// The 2-qubit register on which the operation acts.  
operation PhaseFlip_AllZeros(register : Qubit[]) : Unit is Adj + Ctl {
    use aux = Qubit();
    (ControlledOnInt(0, X))(register, aux);
    Z(aux);
    (ControlledOnInt(0, X))(register, aux);
}

In [ ]:
%%qsharp
/// #Summary
/// The S_chi operation. Applies -1 to the good state i.e., |1>|1> and nothing to others.
/// Can be extended to act on registers of arbitrary length and a specific good state |\Psi>
///
/// #Inputs
/// ## register
/// The 2-qubits register on which the marking operation acts
operation PhaseFlip_Marking(register : Qubit[]) : Unit is Adj + Ctl {
    
    use aux = Qubit();
    within {
        //convert target to |0> - |1>
        X(aux);
        H(aux);
    }
    apply {
        //Get phase kickback for |1>|1> state
        Controlled X (register, aux);
        
    }
}

In [ ]:
%%qsharp
/// #Summary
/// The amplification iterate Q = - U S_0 U^dag S_chi
///
/// #Inputs
/// ## register
/// The 2-qubit register on which the amplification iterate is applied.
operation AmplificationIterate(register : Qubit[]) : Unit is Adj + Ctl {
    PhaseFlip_Marking(register); //S_chi
    Adjoint ArbitRotation(register); // U^dag    
    PhaseFlip_AllZeros(register); //S_0    
    ArbitRotation(register); // U    
    R(PauliI, 2.0*PI(), register[0]);    // Applies -1 global phase
}

For $\theta^* \in [0, \frac{\pi}{2}]$, and $\ket{\Psi}$ obtained by applying $U$ once, [Brassard et al.](http://arxiv.org/abs/quant-ph/0005055v1) showed that $m$ applications of $\mathbf{Q}$ gives: 
$$ \mathbf{Q}^m :\ket{\Psi} \mapsto \sin ((2m+1) \theta^*) \ket{\Psi_1} \ket{1} + \cos ((2m+1) \theta^*) \ket{\Psi_0} \ket{0} \tag{1}$$
Measuring the above state gives $1$ with a probability that is about $4m^2$ larger than measuring $\ket{\Psi}$ with only $2m$ more queries to $U$. This intuitively leads to a possible quadratic quantum speedup. The following operation describes how to raise a given unitary to the $m^{\text{th}}$ power.

In [ ]:
%%qsharp
/// #Summary
/// Given a multi-qubit Unitary V, and a power m apply V^m
///
/// #Inputs
/// ## power
/// The integer power to raise the unitary to.
/// ## target
/// The qubit register on which the unitary is to be applied.
/// ## V
/// The unitary operation to be applied.
operation PowUnitary (power : Int, V : (Qubit[] => Unit is Adj + Ctl), target : Qubit[]) : Unit is Adj + Ctl {
    Fact(power >= 0, "power should be a positive integer.");
    for i in 1..power {
        V(target);
    }
}

## Simulating noise

We consider a commonly used noise model: *the depolarizing noise* as considered in [Tanaka et al.](https://link.springer.com/article/10.1007/s11128-021-03215-9) and [Tiron et al.](https://arxiv.org/abs/2012.03348v1). Its action is described as a channel $\mathcal{D}$ on an $n$-qubit density matrix $\rho$ as: 
$$ \mathcal{D} (\rho) = p \rho + (1 - p) \frac{\mathbb{I}}{2^n}$$
Intuitively this means that with probability $(1-p)$, the qubits are depolarized and remained unchanged otherwise. Similarly, the `AmplificationIterate` operation $\mathbf{Q}$ can be described as a channel $\mathcal{Q}$ with:
$$ \mathcal{Q} (\rho) = \mathbf{Q} \rho \mathbf{Q}^\dag$$

As the most commonly repeated part of the amplitude amplification algorithm is the `AmplificationIterate` operation, we consider the interaction of the noise channel $\mathcal{D}$ with the $\mathcal{Q}$ channel to create the noisy amplification iterate $\mathcal{DQ}$ where
$$ \mathcal{DQ}(\rho) = \mathcal{QD}(\rho) = p \mathbf{Q} \rho \mathbf{Q}^\dag + (1 - p) \frac{\mathbb{I}}{2^n}.$$
Extending this to $m$ repetitions of the noisy amplification process gives
$$ (\mathcal{QD})^m(\rho) = \mathcal{D}^m\mathcal{Q}^m(\rho) = p^m \mathbf{Q}^m \rho \mathbf{Q}^{\dag m} + (1 - p^m) \frac{\mathbb{I}}{2^n}. \tag{2}$$
In other words, with probability $p^m$, apply $m$ repetitions of $\mathbf{Q}$, otherwise, the qubits depolarize. This action is obtained by defining the `AmplitudeAmplification` procedure.

In [ ]:
%%qsharp
/// #Summary
/// The quantum operations to apply the noisy amplification channel mvar times and measure the output
///
/// #Inputs
/// ## mvar
/// The power to which the amplification channel DQ is raised to.
///
/// ## depol_prob
/// The probability with which the qubits are depolarized
///
/// #Outputs
/// ## result
operation AmplitudeAmplification(mvar : Int, depol_prob : Double) : Bool { 
    
    let p = 1.0 - depol_prob;    // qubits depolarize with probability depol_prob
    let prob = DrawRandomDouble(0.0, 1.0);

    use qs = Qubit[2]; // initialize qubits
    
    if prob <= PowD(p, IntAsDouble(mvar)) {
        ArbitRotation(qs); // Output: cos theta/2 |00> + sin theta/2 |11>
        PowUnitary(mvar, AmplificationIterate, qs); // Output: cos (2m+1)theta/2 |00> + sin (2m+1)theta/2 |11>
    }
    else {    // if the qubits are depolarized, reset all the qubits. 
        ResetAll(qs);
        H(qs[1]);    // Set the that will be measured to output 1 with probability 0.5 mimicking depolarized behavior
    }
    
    let res = M(qs[1]); //Measure qubit
    
    ResetAll(qs); //release qubits cleanly
    
    let result = ResultAsBool(res);
    return result;
}

## Quantum-classical Amplitude Estimation in noisy systems

The quantum-classical approach to Amplitude Estimation, applies the noisy amplification channel $\mathcal{DQ}$ $m$ times and then measures the output qubit to record a $0$ and $1$ measurement. For angle $\theta^*$, setting $\gamma = - \ln p$, the probability of obtaining a $1$ outcome after $m$ repetitions of the noisy amplitude amplification procedure is $ \frac{1}{2} - \frac{1}{2} e^{-\gamma m} \cos(2 (2m+1)\theta^*)$. Repeating this procedure for $N_{\text{shots}}$ shots where $1$ is observed $N_1$ times and $0$ is observed $N_0$ times (such that $N_0 + N_1 = N_{\text{shots}}$), the likelihood of the unknown angle being $\theta$ is
$$ \Pr(X | \alpha = \theta) := \left[\frac{1}{2} - \frac{1}{2} e^{-\gamma m} \cos(2 (2m+1)\theta)\right]^{N_1} \left[\frac{1}{2} + \frac{1}{2} e^{-\gamma m} \cos(2 (2m+1)\theta)\right]^{N_0}.$$

Taking $K$ independent samples for different values of $m_k$, for $k = 1, \ldots, K$ the likelihood function becomes
$$ L(\theta) = \Pi_{k=1}^K   \left[\frac{1}{2} - \frac{1}{2} e^{-\gamma m_k} \cos(2 (2m_k+1)\theta)\right]^{N_{k,1}} \left[\frac{1}{2} + \frac{1}{2} e^{-\gamma m_k} \cos(2 (2m_k+1)\theta)\right]^{N_{k,0}}.$$
Equivalently, we can use the log-likelihood function and compute 
$$ \ell(\theta) = \sum_{k=1}^K \left[N_{k,1} \log \left(\frac{1}{2} - \frac{1}{2} e^{-\gamma m_k} \cos(2 (2m_k+1)\theta)\right) + N_{k,0} \log \left(\frac{1}{2} + \frac{1}{2} e^{-\gamma m_k} \cos(2 (2m_k+1)\theta)\right)\right].$$

Computing this log-likelihood function required 
$$
N_{\text{shots}}\sum_{k=1}^K (2m_k + 1)
$$
usages of the operator $U$. 

As there is no prior information, initially we assume that any choice for $\theta \in [0, \pi/2]$ could be equally likely. 
So, the log-likelihood function is initialized to be the the all-zero function. 

Additionally, while $\theta$ is a continuous parameter, as we only need to provide an estimate up to a precision of $\varepsilon$, the interval $[0, \pi/2]$ is divided into $\lceil \frac{1}{\varepsilon} \rceil$ discrete parts that then form the possible choices for $\theta$.
This log-likelihood function for these choices is initialized below. 

📝 **Note:** As subsequent code in this sample now focuses on the classical part of the quantum-classical hybrid, it is expressed in Python.

In [ ]:
# #Summary
# Initialize the log likelihood function to log of uniform probability over choices for theta.
#
# #Inputs
# ## eps
# The precision parameter used to discretize the choice of angles in [0, pi/2] 
#
# #Outputs
# ## init_llhood
# The initial log-likelihood function set to a uniform value over choices for theta
def init_log_likelihood(eps):
    if(eps <= 0 or eps > 1):
        raise ValueError("Precision eps should be a number in (0, 1]")
    
    arr_len = math.ceil(1/eps) 
    norm = arr_len*eps
    init_llhood = np.full(arr_len, 0.0)
    
    return init_llhood


The Bayesian update for the log-likelihood function at each iteration is computed as
$$ \ell_{\text{new}}(\theta) = \ell_{\text{old}}(\theta) + N_{k,1} \log \left(\frac{1}{2} - \frac{1}{2} e^{-\gamma m_k} \cos(2 (2m_k+1)\theta)\right) + N_{k,0} \log \left(\frac{1}{2} + \frac{1}{2} e^{-\gamma m_k} \cos(2 (2m_k+1)\theta)\right) \tag{3}$$
which is described in the method below.

**Note:** We have to be careful to deal with the exceptions when we deal with the logarithm of $0$ here.

In [ ]:
# #Summary
# Compute the Bayesian update of the likelihood function for the current iteration as per Equation (2)
#
# #Inputs
# ## eps 
# The precision parameter
# 
# ## m 
# The power to which the amplitude amplification iterate was raised to
# 
# ## N0 
# The number of 0 outcomes observed for this iteration
# 
# ## N1 
# The number of 1 outcomes observed for this iteration
# 
# ## llhood 
# An array holding the previous values of the log likelihood function for each choice of theta where for index t we have theta_t = pi*t*eps/2
#
# ## depol
# Probability with which depolarizing noise is applied to the qubits
#
# #Output
# ## llhood 
# Return the updated value of the log-likelihood function

def update_log_likelihood(eps, m, N0, N1, llhood, depol):
    length = len(llhood)
    one_val=0
    zero_val=0

    p = 1 - depol 
    
    if(length != math.ceil(1/eps)):
        raise IndexError("Array length mismatch with expected value.")

    for t in range(length):
        val = llhood[t]
        angle = 2*(2*m+1)*t*math.pi*eps/2
        cosval = math.cos(angle)    # cos (2 (2m+1) theta)
        if m == 0:    # to deal with the classical schedule when m is 0
            expval = 1
        else:
            expval = math.pow(p, m)
        if cosval == expval:    # when the second term in eq. 3 could be -infinity
            if N1 == 0:    # to deal with edge case of 0 * -math.inf
                one_val = 0
            else:
                one_val = -math.inf
        elif cosval == -expval:    # when the third term in eq. 3 could be -infinity
            if N0 == 0:    # to deal with edge case of 0 * -math.inf
                zero_val = 0
            else:
                zero_val = -math.inf
        else:
            one_val = N1*math.log(0.5 - 0.5*cosval*expval)
            zero_val = N0*math.log(0.5 + 0.5*cosval*expval)
        
        llhood[t] = val+one_val+zero_val
    
    return llhood

Finally, the estimate for the angle is determined as 
$$ \hat{\theta} = \arg\max_{\theta} L(\theta) = \arg\max_{\theta} \ell(\theta).$$

This naturally leads to the question of choosing an appropriate $K$ and values for $m_k$ to minimize the depth of the circuits while generating a good estimate for the angle to within a given precision of $\varepsilon$. This is because the circuit depth scales as as the number of oracle calls for one run of the algorithm which is given by $(2m_k + 1)$ for iteration $k$. Since $m_k$ monotonically increases with $k$, the maximum number of oracle calls in the circuits is $(2 m_K + 1)$ [Tiron et al.](https://arxiv.org/abs/2012.03348v1) use a power-law schedule that for parameter $\beta \in (0, 1]$:
1. Makes $N = O(1/\varepsilon^{1+\beta})$ total oracle calls
2. Uses circuits of depth at most $D = O(1/\varepsilon^{1 - \beta})$
3. Sets number of amplifications $m_k = \left\lfloor k^{\tfrac{1 - \beta}{2\beta}} \right\rfloor$ for each iteration $k$

Observe that the possible values $\beta \in (0,1]$ covers the gamut of all possible quantum-classical strategies for amplitude estimation from 
- fully quantum (without Quantum Fourier Transform) at $\beta \leftarrow 0$; to
- purely classical sampling at $\beta = 1$

The Python method below interacts with the Q# code from the previous sections of the notebook to generate one sample for the the power law amplitude estimation algorithm and updates the likelihood function accordingly.

In [ ]:
# #Summary
# Execute one run of the Hybrid Amplitude Estimation algorithm.
#
# #Inputs
# ## angle
# The ideal angle used for the oracle and used here for verification purposes
# 
# ## K
# The number of independent samples used to generate the log-likelihood function
# 
# ## power
# The power to which iteration index k is raised to obtain circuit depth m_k.
# Related to power law parameter beta and defined as (1 - beta)/2*beta
# 
# ## log_likelihood
# An array holding the log-likelihood function for each possible choice for theta
# 
# ## eps
# Accuracy parameter
# 
# ## Nshots
# Number of times Amplitude Amplification is repeated to estimate the likelihood function for each iteration
#
# ## depol
# Probability with which depolarizing noise is applied to the qubits
def amp_est_iteration(K, power, llhood, eps, Nshots, depol):

    queries = 0    # Computed total number of queries made

    for k in range(1, K+1):
    
        N0 = 0
        N1 = 0

        if power == math.inf:    # To execute an exponential schedule i.e., m_k = 2^k
            m = math.floor(pow(2, k))
        elif power == 0:    # To execute a classical schedule i.e., m_k = 0
            m = 0
        else:
            m = math.floor(pow(k, power))
        
        for i in range(Nshots):
            res = AmplitudeAmplification.simulate(mvar=m, depol_prob=depol)    # Apply quantum operations and collect measurement result in res
            if res == False:
                N0 = N0 + 1
            else:
                N1 = N1 + 1
        
        llhood = update_log_likelihood(eps, m, N0, N1, llhood, depol)    # Update log-likelihood function
        
        queries = queries + (2*m+1)*Nshots;    # Calculates total number of queries made in this iteration

        if K > 10:
            interval = math.floor(K/10)
        else:    # Account for K < 10
            interval = 1

        if(k % interval == 0):    # Displaying intermediate estimates at every {K/10}^th iteration
            print(f"Iteration k = {k}:")
            print(f"m_k = {m}, N0_k = {N0}, N1_k = {N1}")
            temp_t = np.argmax(llhood)
            temp_angle = math.pi*temp_t*eps/2
            print(f"Current estimate for \u03B8*: {round(temp_angle, 4)}")    # Printing theta to 4 bits of precision
            
            # Computing additive error of current estimate
            err = (UnknownTHETA() - temp_angle)
            print(f"Additive error of current estimate: {round(err, 5)}\n")    # Printing error to 5 bits of precision

    return llhood, queries

The main method that executes the power law amplitude estimation algorithm is described below.

In [ ]:
# #Summary
# Execute the power law amplitude estimation algorithm for a specific choice of parameters
#
# #Inputs
# ## beta
# Power law parameter
#
# ## Nshots
# Number of amplitude amplification repetitions performed to accurately estimate the likelihood in each iteration
#
# ## eps
# Accuracy parameter
#
# ## depol
# Probability with which depolarizing noise is applied to the qubits
#
# #Outputs
# ## theta_hat
# The final estimate for the unknown angle output by the algorithm

def power_law_amp_est(beta, Nshots, eps, depol):

    if (Nshots <= 0 or not type(Nshots) is int):
        raise ValueError("Nshots should be an integer > 0")
    if(beta < 0 or beta > 1):
        raise ValueError("Power law parameter beta should be a number in [0, 1]")
    if(eps <= 0 or eps > 1):
        raise ValueError("Precision eps should be a number in (0, 1]")
    if(depol < 0 or depol > 1):
        raise ValueError("Probability of depolarizing qubits should be in [0, 1]")
    if(UnknownTHETA() < 0 or UnknownTHETA() > math.pi/2):
        raise ValueError("Unknown angle in UnknownTHETA() should be in [0, pi/2]")
    
    log_likelihood = init_log_likelihood(eps)    # Initialize distribution
    N0=0
    N1=0
    queries=0
    queries_expected=0
    
    K = math.ceil(max(math.log(1/eps), 1/pow(eps, 2*beta)))    # Compute K 

    if beta == 0:    # Accounting for the edge case of executing an exponential schedule
        power = math.inf
        max_depth = 2*math.floor(pow(2, K))+1
        for k in range(1, K+1):
            queries_expected= queries_expected + Nshots*(2*math.floor(pow(2, k))+1)
    elif beta == 1:    # Accounting for the edge case of executing a classical schedule
        max_depth = 1
        power = 0
        for k in range(1, K+1):
            queries_expected= queries_expected + Nshots*(max_depth)
    else:
        power = (1 - beta)/(2*beta)
        max_depth = math.floor(pow(K, power)) 
        for k in range(1, K+1):
            queries_expected= queries_expected + Nshots*(2*math.floor(pow(k, power))+1)
    
    length = len(log_likelihood)
    print("\033[1mAlgorithm Metrics \033[0m")
    print(f"Unknown angle \u03B8*: {round(UnknownTHETA(), 4)}")
    print(f"Number of samples K: {K}")
    
    if(beta == 0):
        print("Schedule: Exponential, i.e. m_k = 2^k, for k = 1,...,K")
    elif (beta == 1):
        print("Schedule: Classical i.e. m_k = 0 for k = 1,...,K")
    elif round(beta-1/3, 4) == 0:
        print("Schedule: Linear, i.e. m_k = k, for k = 1,...,K")
    else:
        print(f"Schedule: Power law with \u03B2 = {round(beta, 4)}, i.e. m_k = \u230Ak^((1 - \u03B2)/2\u03B2)\u230B = \u230Ak^{round((1 - beta)/(2*beta),4)}\u230B, for k = 1,...,{K}")

    print(f"Maximum depth: O(2 m_K + 1); and (2 m_K + 1) = {max_depth}")
    print(f"Resolution of choices for \u03B8*: {length}")
    print(f"Number of oracle calls: {queries_expected}")
    print(f"Probability of depolarizing qubits: {depol}\n")
    
    print("\033[1mIntermediate Estimates \033[0m")
    log_likelihood, queries = amp_est_iteration(K, power, log_likelihood, eps, Nshots, depol)
    
    # Compute final estimate    
    print("\n\033[1mFinal estimates \033[0m")
    t = np.argmax(log_likelihood)
    theta_hat = math.pi*t*eps/2
    print(f"Unknown angle \u03B8*: {round(UnknownTHETA(), 4)}") 
    print(f"Estimate for \u03B8*: {round(theta_hat, 4)}")    # Printing to 4 bits of precision.
    print(f"Total number of oracle calls: {queries}")
    
    # Compute additive error 
    # The estimate is within precision limits if additive error is in [-eps, +eps].
    err = (UnknownTHETA() - theta_hat)
    
    print(f"Additive error of final estimate of \u03B8*: {round(err, 5)}")    # Printing error to 5 bits of precision

    return theta_hat

## Examples

First, we execute the algorithm for an exponential schedule with $\beta = 0$, $N_{\text{shots}} = 50$, $\varepsilon = 0.005$ and $20\%$ depolarizing noise.

In [ ]:
# Executing the exponential schedule with 20% depolarizing noise
theta_hat = power_law_amp_est(beta=0, Nshots=50, eps=0.005, depol = 0.2)

The key observation here is that the depolarizing noise is such that the exponential schedule does not provide an accurate estimate for $\theta^*$. 

📝 **Remark:** It is possible that one run of the cell above does give an accurate estimate of $\theta^*$. However, running the cell multiple times, one can observe that this is a rare occurrence. This can be attributed to the probabilistic nature of the noise model considered. In particular, in the *rare case* when there is almost no noise present in a run, the exponential schedule gives a good estimate.

To overcome this uncertainty in getting a good estimate, we instead try the power law schedule with $\beta = 0.5$.

🛑 The computation below can take up to a minute to complete.

In [ ]:
theta_hat = power_law_amp_est(beta=0.5, Nshots=50, eps=0.005, depol=0.2)

📝 **Remark:** By running the above cell multiple observe that, with very high probability, each run gives an accurate estimate of $\theta^*$.

From these two runs, we can draw the following conclusion. Although the exponential schedule has fewer samples, the higher depth circuit in the presence of noise in unable to provide an accurate estimate for $\theta^*$. On the other hand, although the power law schedule require many more samples and oracle calls, the relatively low depth of the circuits helps one obtain an accurate estimate for $\theta^*$.

Further, we can back up the need for the hybrid approach of the power law schedule by contrasting its requirements (in terms of samples and oracle calls) with that of a fully classical sampling approach where $\beta = 1$. As the full execution of this classical schedule (with all other parameters unchanged) will take more than 100 times longer than the previous power law schedule, we only give the algorithm metrics for this case below.

**Algorithm Metrics**
```
Unknown angle θ*:0.9708
Number of samples K: 40000
Schedule: Classical i.e. m_k = 0 for k = 1,...,K
Maximum depth: O(2 m_K + 1); and (2 m_K + 1) = 1
Resolution of choices for θ*: 200
Number of oracle calls: 2000000
Probability of depolarizing qubits: 0.2
```

## Next steps

This notebook uses a simple oracle to demonstrate the effectiveness of hybrid classical and quantum computing to overcome the challenges faced in noisy intermediate scale systems. Feel free to use this as a starting point for your experiments and further exploration. For example, you can 

- explore how the results change for different oracles and different input parameters;
- deploy this code on a QPU to explore if the hardware results match the above simulations;
- perform resource estimation to get a break down of resources needed under different assumptions; 